# Flutter Localization Synchronizer

A utility notebook to synchronize variables between `app_localizations_en.dart` and `app_localizations.dart` files in Flutter projects.

## Import Required Libraries

Import libraries such as os, re, and json for file handling and parsing.

In [ ]:
import os
import re
import json
import difflib
from pathlib import Path

print("Libraries imported successfully")

## Load Localization Files

Read the contents of `app_localizations_en.dart` and `app_localizations.dart` into memory.

In [ ]:
# Define file paths - adjust these paths to match your Flutter project structure
base_path = Path("path/to/your/flutter/project")
en_file_path = base_path / "lib" / "l10n" / "app_localizations_en.dart"
base_file_path = base_path / "lib" / "l10n" / "app_localizations.dart"

# Function to read a file's content
def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error reading file {file_path}: {str(e)}")
        return None

# Read the localization files
en_content = read_file(en_file_path)
base_content = read_file(base_file_path)

if en_content and base_content:
    print("Files read successfully")
    # Display first 200 characters of each file as a preview
    print("\nEnglish localization file preview:")
    print(en_content[:200] + "...\n")
    print("Base localization file preview:")
    print(base_content[:200] + "...")
else:
    print("Failed to read one or both files. Please check the file paths.")

## Parse Variables in `app_localizations_en.dart`

Extract all variable names and their values from `app_localizations_en.dart` using regular expressions.

In [ ]:
def extract_variables(content):
    """
    Extract variable names and their string values from a Dart localization file
    """
    if not content:
        return {}
    
    # Regular expression to match Dart string variable declarations
    # This pattern looks for lines like: String get variableName => 'some value';
    pattern = r"String\s+get\s+(\w+)\s*=>\s*[\"\'](.*?)[\"\'];"
    
    # Find all matches
    matches = re.findall(pattern, content)
    
    # Create a dictionary of variable names and values
    variables = {name: value for name, value in matches}
    
    return variables

# Extract variables from the English localization file
en_variables = extract_variables(en_content)

# Display the extracted variables
print(f"Found {len(en_variables)} variables in the English localization file:")
for i, (name, value) in enumerate(list(en_variables.items())[:10]):
    print(f"{name}: {value}")
    
if len(en_variables) > 10:
    print(f"... and {len(en_variables) - 10} more")

## Compare and Synchronize with `app_localizations.dart`

Identify missing variables in `app_localizations.dart` and prepare them for addition.

In [ ]:
# Extract variables from the base localization file
base_variables = extract_variables(base_content)

# Find variables that exist in English but not in the base file
missing_variables = {name: value for name, value in en_variables.items() 
                    if name not in base_variables}

# Find variables that exist in both files but have different values
different_values = {name: (base_variables[name], en_variables[name]) 
                   for name in set(base_variables.keys()) & set(en_variables.keys())
                   if base_variables[name] != en_variables[name]}

# Find variables that exist in base but not in English (may be deprecated)
extra_variables = {name: value for name, value in base_variables.items() 
                  if name not in en_variables}

# Display results
print(f"Missing variables (in English but not in base): {len(missing_variables)}")
for name, value in list(missing_variables.items())[:5]:
    print(f"  {name}: '{value}'")
if len(missing_variables) > 5:
    print(f"  ... and {len(missing_variables) - 5} more")

print(f"\nVariables with different values: {len(different_values)}")
for name, (base_value, en_value) in list(different_values.items())[:5]:
    print(f"  {name}:")
    print(f"    Base: '{base_value}'")
    print(f"    English: '{en_value}'")
if len(different_values) > 5:
    print(f"  ... and {len(different_values) - 5} more")

print(f"\nExtra variables (in base but not in English): {len(extra_variables)}")
for name, value in list(extra_variables.items())[:5]:
    print(f"  {name}: '{value}'")
if len(extra_variables) > 5:
    print(f"  ... and {len(extra_variables) - 5} more")

## Write Updated `app_localizations.dart`

Write the updated content back to `app_localizations.dart`, ensuring all variables are synchronized.

In [ ]:
def update_base_file(base_content, missing_variables):
    """
    Update the base localization file by adding missing variables
    """
    if not base_content or not missing_variables:
        return base_content
    
    # Find the position to insert new variables (before the closing brace of the class)
    class_end_pattern = r"}\s*$"
    match = re.search(class_end_pattern, base_content, re.MULTILINE | re.DOTALL)
    
    if not match:
        print("Could not find the end of the class in the base file")
        return base_content
    
    insert_position = match.start()
    
    # Create the text to insert
    new_variables_text = "\n\n  // Added automatically by localization synchronizer\n"
    for name, value in missing_variables.items():
        new_variables_text += f"  String get {name} => '{value}';\n"
    
    # Insert the new variables
    updated_content = base_content[:insert_position] + new_variables_text + base_content[insert_position:]
    
    return updated_content

# Update the base file content
updated_base_content = update_base_file(base_content, missing_variables)

# Show a preview of the changes
print("Preview of changes:")
if base_content == updated_base_content:
    print("No changes were made to the base file.")
else:
    # Show diff of the changes
    diff = difflib.unified_diff(
        base_content.splitlines(keepends=True),
        updated_base_content.splitlines(keepends=True),
        fromfile='app_localizations.dart (original)',
        tofile='app_localizations.dart (updated)',
        n=3
    )
    print(''.join(list(diff)[:20]))
    print("...")

In [ ]:
# Write the updated content to the base file
def write_file(file_path, content):
    """
    Write content to a file
    """
    try:
        # Make a backup of the original file
        backup_path = str(file_path) + '.bak'
        with open(file_path, 'r', encoding='utf-8') as original:
            with open(backup_path, 'w', encoding='utf-8') as backup:
                backup.write(original.read())
        
        # Write the updated content
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        
        return True
    except Exception as e:
        print(f"Error writing to file {file_path}: {str(e)}")
        return False

# Function to actually perform the update (commented out for safety)
def perform_update():
    if base_content == updated_base_content:
        print("No updates needed. The files are already synchronized.")
        return
    
    # Confirm before writing
    confirmation = input("Do you want to write the updated content to app_localizations.dart? (yes/no): ")
    if confirmation.lower() == 'yes':
        success = write_file(base_file_path, updated_base_content)
        if success:
            print(f"Successfully updated {base_file_path}")
            print(f"A backup of the original file was saved to {base_file_path}.bak")
        else:
            print("Failed to update the file. See error message above.")
    else:
        print("Update canceled.")

# Uncomment the line below to perform the update
# perform_update()

print("To perform the update, uncomment the call to perform_update() in the cell above and run it again.")

## Summary

This notebook has:
1. Imported necessary libraries for file handling and text processing
2. Loaded the localization files from your Flutter project
3. Extracted variables from both files using regular expressions
4. Identified missing variables that need to be synchronized
5. Prepared an updated version of `app_localizations.dart`
6. Provided a function to write the changes back to the file (with backup)

To use this notebook effectively:
1. Update the file paths to match your Flutter project structure
2. Run each cell in sequence
3. Review the missing variables before applying changes
4. Uncomment the `perform_update()` call when you're ready to write changes

This ensures your localization files stay synchronized, making your Flutter app's internationalization consistent across all languages.

# Localization Variables Synchronization Tool

This notebook provides a tool to synchronize variables between `app_localizations_en.dart` and `app_localizations.dart` in a Flutter project. It identifies variables present in the English localization file but missing in the base localization file and adds them automatically.

## Import Required Libraries
Import necessary libraries such as os, re, and json for file handling and string processing.

In [ ]:
import os
import re
import json
import difflib
from pathlib import Path

## Load Localization Files
Read the contents of app_localizations_en.dart and app_localizations.dart into memory.

In [ ]:
# Define the paths to the localization files
# Change these paths to match your project structure
project_root = Path("../")  # Adjust if needed
en_file_path = project_root / "lib" / "l10n" / "app_localizations_en.dart"
base_file_path = project_root / "lib" / "l10n" / "app_localizations.dart"

# Function to read file content
def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

# Read the content of both files
en_content = read_file(en_file_path)
base_content = read_file(base_file_path)

if en_content and base_content:
    print("Both files loaded successfully!")
    print(f"EN file size: {len(en_content)} characters")
    print(f"Base file size: {len(base_content)} characters")
else:
    print("Failed to load one or both files.")

## Extract Variables from app_localizations_en.dart
Use regular expressions to extract all variable names and their values from app_localizations_en.dart.

In [ ]:
def extract_variables(content):
    """
    Extract localization variables and their values from dart file content
    Returns a dictionary with variable names as keys and their values as values
    """
    # Regex to match string getters in the dart file
    pattern = r'String get (\w+)\s*=>\s*[\'"](.+?)[\'"];'
    matches = re.findall(pattern, content)
    
    variables = {}
    for var_name, var_value in matches:
        variables[var_name] = var_value
    
    return variables

# Extract variables from both files
en_variables = extract_variables(en_content)
base_variables = extract_variables(base_content)

print(f"Found {len(en_variables)} variables in the English localization file")
print(f"Found {len(base_variables)} variables in the base localization file")

# Display some examples of extracted variables
print("\nSample variables from English file:")
sample_keys = list(en_variables.keys())[:5]  # First 5 keys
for key in sample_keys:
    print(f"  {key}: {en_variables[key]}")

## Compare Variables with app_localizations.dart
Identify variables present in app_localizations_en.dart but missing in app_localizations.dart.

In [ ]:
# Find variables in en file but missing in base file
missing_variables = {key: en_variables[key] for key in en_variables if key not in base_variables}

print(f"Found {len(missing_variables)} variables missing in the base localization file:")
for key, value in missing_variables.items():
    print(f"  {key}: {value}")

# Find variables that exist in both files but might have different values
common_variables = {key: (en_variables[key], base_variables[key]) 
                   for key in en_variables if key in base_variables and en_variables[key] != base_variables[key]}

print(f"\nFound {len(common_variables)} variables with different values:")
for key, (en_value, base_value) in common_variables.items():
    print(f"  {key}:")
    print(f"    EN: {en_value}")
    print(f"    Base: {base_value}")

## Synchronize Missing Variables
Add the missing variables from app_localizations_en.dart to app_localizations.dart.

In [ ]:
def add_missing_variables(content, missing_vars):
    """
    Add missing variables to the content while preserving the structure
    """
    # Find the position of the last getter in the class
    last_getter_pattern = r'String get \w+\s*=>\s*[\'"].+?[\'"];'
    matches = list(re.finditer(last_getter_pattern, content))
    
    if not matches:
        print("Could not find a suitable position to insert the new variables")
        return content
    
    last_match = matches[-1]
    insert_position = last_match.end()
    
    # Prepare new variables to insert
    new_content = []
    for var_name, var_value in missing_vars.items():
        new_content.append(f"\n  String get {var_name} => \"{var_value}\";")
    
    # Insert the new variables
    updated_content = content[:insert_position] + ''.join(new_content) + content[insert_position:]
    return updated_content

# Create updated content
updated_base_content = add_missing_variables(base_content, missing_variables)

# Show a preview of changes
if base_content != updated_base_content:
    print("Changes to be made to the base localization file:")
    
    # Show diff of the changes
    diff = difflib.unified_diff(
        base_content.splitlines(),
        updated_base_content.splitlines(),
        lineterm='',
        n=3  # Context lines
    )
    
    for line in diff:
        if line.startswith('+') and not line.startswith('+++'):
            print(f"\033[92m{line}\033[0m")  # Green for additions
        elif line.startswith('-') and not line.startswith('---'):
            print(f"\033[91m{line}\033[0m")  # Red for deletions
        else:
            print(line)
else:
    print("No changes needed to the base file.")

## Save Updated app_localizations.dart
Write the updated content back to app_localizations.dart, ensuring proper formatting.

In [ ]:
def save_file(file_path, content):
    """
    Save the content to the file
    """
    try:
        # Create a backup of the original file
        backup_path = str(file_path) + ".bak"
        if os.path.exists(file_path):
            with open(backup_path, 'w', encoding='utf-8') as backup_file:
                with open(file_path, 'r', encoding='utf-8') as original_file:
                    backup_file.write(original_file.read())
            print(f"Backup created at {backup_path}")
        
        # Write the updated content
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        print(f"Successfully updated {file_path}")
        return True
    except Exception as e:
        print(f"Error saving file: {e}")
        return False

# Ask for confirmation before saving
confirmation = input("Do you want to save the changes to app_localizations.dart? (yes/no): ")

if confirmation.lower() in ['yes', 'y']:
    save_file(base_file_path, updated_base_content)
    print("Changes saved successfully!")
else:
    print("Operation cancelled. No changes were made.")

## Summary

This notebook has:
1. Loaded the localization files
2. Extracted variables from both files
3. Identified missing variables in the base file
4. Created updated content with the missing variables added
5. Saved the changes (with your confirmation)

You can run this notebook whenever you add new localization strings to ensure your base localization file stays in sync with your English localization file.

# Flutter Localization Synchronizer

This notebook provides a tool to synchronize variables between `app_localizations_en.dart` and `app_localizations.dart` files in a Flutter project. It ensures that all localization variables defined in the English version are also available in other language versions.

## Import Required Libraries

Import necessary libraries such as os, re, and json for file handling and string processing.

In [ ]:
import os
import re
import json
import difflib
from pathlib import Path

## Load Localization Files

Read the contents of app_localizations_en.dart and app_localizations.dart into memory.

In [ ]:
# Function to read dart files
def read_dart_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# Set file paths - update these to match your project structure
project_dir = Path(".")  # Current directory, update as needed
en_file_path = project_dir / "lib" / "l10n" / "app_localizations_en.dart"
base_file_path = project_dir / "lib" / "l10n" / "app_localizations.dart"

# Read the files
en_content = read_dart_file(en_file_path)
base_content = read_dart_file(base_file_path)

if en_content and base_content:
    print(f"Successfully loaded both localization files.")
    print(f"English file size: {len(en_content)} characters")
    print(f"Base file size: {len(base_content)} characters")
else:
    print("Failed to load one or both files. Check the file paths.")

## Extract Variables from app_localizations_en.dart

Use regular expressions to extract all variable names and their values from app_localizations_en.dart.

In [ ]:
def extract_localization_variables(content):
    """Extract variable declarations from a localization dart file."""
    # Pattern to match variable declarations like: String get variableName => 'value';
    pattern = r'String\s+get\s+(\w+)\s*=>\s*[\'"](.+?)[\'"]\s*;'
    
    # Find all matches
    matches = re.findall(pattern, content)
    
    # Convert to dictionary {variable_name: value}
    variables = {name: value for name, value in matches}
    
    return variables

# Extract variables from the English localization file
if en_content:
    en_variables = extract_localization_variables(en_content)
    print(f"Found {len(en_variables)} variables in app_localizations_en.dart")
    
    # Display a few examples
    sample_count = min(5, len(en_variables))
    print(f"\nSample of {sample_count} variables:")
    for i, (name, value) in enumerate(list(en_variables.items())[:sample_count]):
        print(f"{i+1}. {name} => '{value}'")

## Compare Variables with app_localizations.dart

Identify variables present in app_localizations_en.dart but missing in app_localizations.dart.

In [ ]:
# Extract variables from the base localization file
if base_content:
    base_variables = extract_localization_variables(base_content)
    print(f"Found {len(base_variables)} variables in app_localizations.dart")
    
    # Find missing variables
    missing_variables = {name: value for name, value in en_variables.items() 
                         if name not in base_variables}
    
    print(f"\nFound {len(missing_variables)} variables missing in app_localizations.dart")
    
    if missing_variables:
        print("\nMissing variables:")
        for i, (name, value) in enumerate(missing_variables.items()):
            print(f"{i+1}. {name} => '{value}'")
    else:
        print("All variables are synchronized!")

## Synchronize Missing Variables

Add missing variables from app_localizations_en.dart to app_localizations.dart while maintaining the file's structure.

In [ ]:
def add_missing_variables(content, missing_vars):
    """Add missing variables to the localization file content."""
    if not missing_vars:
        return content
    
    # Find where to insert the new variables
    # Usually before the closing bracket of the class
    match = re.search(r'(\s*})(\s*)$', content)
    if not match:
        print("Could not find a suitable position to insert variables")
        return content
    
    insert_position = match.start()
    
    # Format new variables
    new_variables = []
    for name, value in missing_vars.items():
        # For non-English localization, we keep the English value as a comment
        new_variables.append(f"  String get {name} => '{value}'; // TODO: Translate")
    
    # Insert the new variables
    new_content = (
        content[:insert_position] + 
        "\n  // Variables added by localization_sync.ipynb\n" + 
        "\n".join(new_variables) + 
        "\n" + 
        content[insert_position:]
    )
    
    return new_content

# Update the content with missing variables
if base_content and missing_variables:
    updated_content = add_missing_variables(base_content, missing_variables)
    
    # Show a diff of the changes (optional)
    diff = difflib.unified_diff(
        base_content.splitlines(),
        updated_content.splitlines(),
        lineterm='',
        n=1
    )
    print("Changes to be made:")
    for line in diff:
        print(line)

## Save Updated app_localizations.dart

Write the updated content back to app_localizations.dart.

In [ ]:
def save_dart_file(file_path, content):
    """Save content to a dart file."""
    try:
        # Create a backup first
        backup_path = str(file_path) + '.bak'
        if os.path.exists(file_path):
            with open(backup_path, 'w', encoding='utf-8') as backup_file:
                with open(file_path, 'r', encoding='utf-8') as original_file:
                    backup_file.write(original_file.read())
            print(f"Created backup at {backup_path}")
        
        # Write the updated content
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        return True
    except Exception as e:
        print(f"Error saving file {file_path}: {e}")
        return False

# Confirm and save changes
if base_content and missing_variables:
    # You can uncomment this to actually save the file
    # (disabled by default for safety)
    
    # confirmation = input("Save changes to app_localizations.dart? (yes/no): ")
    # if confirmation.lower() in ['yes', 'y']:
    #     if save_dart_file(base_file_path, updated_content):
    #         print(f"Successfully updated {base_file_path}")
    #     else:
    #         print("Failed to save changes.")
    # else:
    #     print("Changes not saved.")
    
    print("To save changes, uncomment the confirmation code in this cell")

## Conclusion

This notebook provides a simple way to synchronize localization variables between dart files in a Flutter project. You can run it whenever you add new strings to your English localization file to ensure all language files stay in sync.

You may want to extend this tool to:
1. Handle more complex string formats (multiline strings, string interpolation)
2. Process multiple language files at once
3. Add options for backup and restore
4. Create a command-line version for integration into development workflows